In [31]:
import json
import pandas as pd
import snowflake.connector as sf
import os
import xlwings as xw
from xlwings.constants import DeleteShiftDirection
import datetime
import numpy as np
pd.options.display.float_format = '{:.2f}'.format

##########################' 
# Analyst fill
vendor_name = 'ABC'
analyst_name = 'DN'
date_batch = '20210801'
#######################

config_coles = r"config.json"
config_coles_clean = r"config2.json"

file_sql_summ_state = r"summarizer_state.sql"
file_sql_summ_state_no_promo = r"summarizer_state_no_promo.sql"
file_sql_cd_ref = r"cd_ref.sql"
file_sql_dept = r"dept.sql"
file_sql_gst = r"gst.sql"
file_sql_cd_ref_listagg = r"cd_ref_listagg.sql"
file_sql_cd_ref_listagg_item = r"cd_ref_listagg_item.sql"
file_sql_pct = r"count_pct.sql"


os.chdir('D:\\python\\ce_scan_summarizer')

path_excel = r"CS_SCAN_Vendorname_Analyst_Date.xlsx"
path_import_item = 'item_import.xlsx'

path_export_final = 'CS_SCAN_'+vendor_name+'_'+analyst_name+'_'+date_batch+'.xlsx'
path_export_final_xlsb = 'CS_SCAN_'+vendor_name+'_'+analyst_name+'_'+date_batch+'.xlsb'

def set_up(config):
    """Set up connection to SnowFlake"""
    config = json.loads(open(config).read())
    account = config['snowflake']['account']
    user = config['snowflake']['user']
    warehouse = config['snowflake']['warehouse']
    role = config['snowflake']['role']
    database = config['snowflake']['database']
    schema = config['snowflake']['schema']
    password = config['snowflake']['password']
    auth = config['snowflake']['authenticator']

    conn = sf.connect(user=user, password=password, account=account, authenticator=auth,
                      warehouse=warehouse, role=role, database=database, schema=schema)

    cursor = conn.cursor()
    return cursor
def connect_sql(cursor,file_sql,item_code,var_1 = '',var_2='',var_3='',var_4 = '',var_5 = '',var_6=''):
    try:
        cursor.execute((open(file_sql).read()).format(item_code,var_1,var_2,var_3,var_4,var_5,var_6))
        all_rows = cursor.fetchall()
        field_names = [i[0] for i in cursor.description]
    finally:
        # conn.close()
        pass
    df = pd.DataFrame(all_rows)
    try:
        df.columns = field_names
    except ValueError:
        return pd.DataFrame(columns=field_names)
    return df

def convert_to_input_sql(num_list):
    num_list_final = ''
    # print('SUPP LIST',supp_num_list)
    for num_list in num_list:
        num_list_final = num_list_final + "'" + num_list + "',"
    return num_list_final[:-1]

def convert_to_input_function(num_list):
    num_list_final = ''
    # print('SUPP LIST',supp_num_list)
    for num_list in num_list:
        num_list_final = num_list_final + num_list + ','
    return num_list_final[:-1]
   

In [23]:
cursor = set_up(config = config_coles)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [40]:
dict_import = {}
excel_file = pd.ExcelFile(path_import_item)
sheet_names = excel_file.sheet_names
for sheet_name in sheet_names:
    df = pd.read_excel(path_import_item,sheet_name=str(sheet_name))
    dict_import[sheet_name] = df
excel_file.close()
dict_import

{'1':        ITEM_IDNT CLASSIFY_STATE     STATE CLASSIFY_PROMO  CLM_START  \
 0  4001686153116       NATIONAL  NATIONAL             NO 2021-08-05   
 1  4001686305409       NATIONAL  NATIONAL             NO 2021-08-05   
 2  4001686340639       NATIONAL  NATIONAL             NO 2021-08-05   
 
      CLM_END  PROMO_ID  RRP  SCANRATE  PERCENTAGE  \
 0 2021-09-01     42575    4      0.20         NaN   
 1 2021-09-01     42575    5      0.20         NaN   
 2 2021-09-01     42575    4      0.20         NaN   
 
                                           EXCEL_PATH  \
 0  https://myprofectus.sharepoint.com/:f:/r/sites...   
 1  https://myprofectus.sharepoint.com/:f:/r/sites...   
 2  https://myprofectus.sharepoint.com/:f:/r/sites...   
 
                                           EMAIL_PATH  
 0  https://myprofectus.sharepoint.com/:f:/r/sites...  
 1  https://myprofectus.sharepoint.com/:f:/r/sites...  
 2  https://myprofectus.sharepoint.com/:f:/r/sites...  ,
 '2':         ITEM_IDNT CLASSIFY

In [41]:
i = 1
df = dict_import[str(i)]
df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df['ITEM_IDNT'] = df['ITEM_IDNT'].astype(str)
df['ITEM_IDNT'] = df['ITEM_IDNT'].str.strip() 
df

,ITEM_IDNT,CLASSIFY_STATE,STATE,CLASSIFY_PROMO,CLM_START,CLM_END,PROMO_ID,RRP,SCANRATE,PERCENTAGE,EXCEL_PATH,EMAIL_PATH
0,4001686153116,NATIONAL,NATIONAL,NO,2021-08-05,2021-09-01,42575,4,0.20,NaN,https://myprofectus.sharepoint.com/:f:/r/sites...,https://myprofectus.sharepoint.com/:f:/r/sites...
1,4001686305409,NATIONAL,NATIONAL,NO,2021-08-05,2021-09-01,42575,5,0.20,NaN,https://myprofectus.sharepoint.com/:f:/r/sites...,https://myprofectus.sharepoint.com/:f:/r/sites...
2,4001686340639,NATIONAL,NATIONAL,NO,2021-08-05,2021-09-01,42575,4,0.20,NaN,https://myprofectus.sharepoint.com/:f:/r/sites...,https://myprofectus.sharepoint.com/:f:/r/sites...


In [42]:
item_unique = df['ITEM_IDNT'].drop_duplicates().tolist()
item_unique = "','".join(item_unique)
clm_start = df['CLM_START'][0]
clm_end = df['CLM_END'][0]
df_gst = connect_sql(cursor,file_sql=file_sql_gst, item_code=item_unique)
gst = df_gst['CML_COST_GST_RATE_PCT'][0]
gst = int(gst)
claim_number = f'{i}_{gst}'
dept = df_gst['DEPT_IDNT'][0]
supp_num = df_gst['SUPP_IDNT'][0]
supp_desc = df_gst['SUPP_DESC'][0]
vendor_num = df_gst['VENDOR_NUM'][0]
classify_state = df['CLASSIFY_STATE'][0]
file_path_excel = df['EXCEL_PATH'][0]
file_path_email = df['EMAIL_PATH'][0]
pct = df['PERCENTAGE'][0]
if np.isnan(pct) == False:
    df_pct = connect_sql(cursor, file_sql_pct, item_code=item_unique, start_date= clm_start , end_date= clm_end)
    df_pct['ITEM_IDNT'] = df_pct['ITEM_IDNT'].astype(str)
    df_pct['ITEM_IDNT'] = df_pct['ITEM_IDNT'].str.strip()
    df = df.merge( right= df_pct , how = 'left',on ='ITEM_IDNT')
    df['RRP'] = (100- df['PERCENTAGE'].astype('float') )/ 100 * df['NORMAL_PRICE'].astype('float')
else:
    pass
df

,ITEM_IDNT,CLASSIFY_STATE,STATE,CLASSIFY_PROMO,CLM_START,CLM_END,PROMO_ID,RRP,SCANRATE,PERCENTAGE,EXCEL_PATH,EMAIL_PATH
0,4001686153116,NATIONAL,NATIONAL,NO,2021-08-05,2021-09-01,42575,4,0.20,NaN,https://myprofectus.sharepoint.com/:f:/r/sites...,https://myprofectus.sharepoint.com/:f:/r/sites...
1,4001686305409,NATIONAL,NATIONAL,NO,2021-08-05,2021-09-01,42575,5,0.20,NaN,https://myprofectus.sharepoint.com/:f:/r/sites...,https://myprofectus.sharepoint.com/:f:/r/sites...
2,4001686340639,NATIONAL,NATIONAL,NO,2021-08-05,2021-09-01,42575,4,0.20,NaN,https://myprofectus.sharepoint.com/:f:/r/sites...,https://myprofectus.sharepoint.com/:f:/r/sites...


In [43]:
item_unique = df['ITEM_IDNT'].drop_duplicates().tolist()
item_unique = "','".join(item_unique)
clm_start = df['CLM_START'][0]
clm_end = df['CLM_END'][0]
df_gst = connect_sql(cursor,file_sql=file_sql_gst, item_code=item_unique)
gst = df_gst['CML_COST_GST_RATE_PCT'][0]
gst = int(gst)
claim_number = f'{i}_{gst}'
dept = df_gst['DEPT_IDNT'][0]
supp_num = df_gst['SUPP_IDNT'][0]
supp_desc = df_gst['SUPP_DESC'][0]
vendor_num = df_gst['VENDOR_NUM'][0]
classify_state = df['CLASSIFY_STATE'][0]
file_path_excel = df['EXCEL_PATH'][0]
file_path_email = df['EMAIL_PATH'][0]
pct = df['PERCENTAGE'][0]
classify_promo = df['CLASSIFY_PROMO'][0]
if classify_promo.lower() == 'yes':
    promo_id = df['PROMO_ID'][0]
else:
    promo_id = ''
if np.isnan(pct) == False:
    df_pct = connect_sql(cursor, file_sql_pct, item_code=item_unique, start_date= clm_start , end_date= clm_end)
    df_pct['ITEM_IDNT'] = df_pct['ITEM_IDNT'].astype(str)
    df_pct['ITEM_IDNT'] = df_pct['ITEM_IDNT'].str.strip()
    df = df.merge( right= df_pct , how = 'left',on ='ITEM_IDNT')
    df['RRP'] = (100- df['PERCENTAGE'].astype('float') )/ 100 * df['NORMAL_PRICE'].astype('float')
else:
    pass
if classify_state.lower() == 'state':
    # item_list_dict = df.set_index(['ITEM_IDNT','STATE'])[['RRP','SCANRATE']].to_dict('index')
    df = df[['ITEM_IDNT','STATE','RRP','SCANRATE']].drop_duplicates()
    df['SCANRATE'] = df['SCANRATE'].round(2)
    df['RRP'] = df['RRP'].round(2)
    df = df.groupby(by = ['STATE','RRP','SCANRATE'])['ITEM_IDNT'].agg(list).to_frame().reset_index()
    df['ITEM_IDNT'] = df['ITEM_IDNT'].apply(lambda x : convert_to_input_function(x))
    df = df.groupby(by = ['RRP','SCANRATE','ITEM_IDNT'])['STATE'].agg(list).to_frame().reset_index()
    df['STATE'] = df['STATE'].apply(lambda x : convert_to_input_sql(x))
    item_list_dict = df.set_index(['ITEM_IDNT','STATE'])[['RRP','SCANRATE']].to_dict('index')
    for key,value in item_list_dict.items():
        item_list_dict[key] = [item_list_dict[key]['RRP']] + [item_list_dict[key]['SCANRATE']] 
    print(df)
    print(item_list_dict)
else:
    # item_list_dict = df.set_index('ITEM_IDNT')[['RRP','SCANRATE']].to_dict('index')
    df = df[['ITEM_IDNT','RRP','SCANRATE']].drop_duplicates()
    df['SCANRATE'] = df['SCANRATE'].round(2)
    df['RRP'] = df['RRP'].round(2)
    df = df.groupby(by = ['RRP','SCANRATE'])['ITEM_IDNT'].agg(list).to_frame().reset_index()
    df['ITEM_IDNT'] = df['ITEM_IDNT'].apply(lambda x : convert_to_input_function(x))
    df['STATE'] = "'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'"
    item_list_dict = df.set_index(['ITEM_IDNT','STATE'])[['RRP','SCANRATE']].to_dict('index')
    for key,value in item_list_dict.items():
        item_list_dict[key] = [item_list_dict[key]['RRP']] + [item_list_dict[key]['SCANRATE']] 
    print(df)
    print(item_list_dict)
for key,value in item_list_dict.items():
    if gst == 10:
        item_list_dict[key][0] = item_list_dict[key][0] /1.1 
    else:
        pass
promo_id

   RRP  SCANRATE                    ITEM_IDNT  \
0    4      0.20  4001686153116,4001686340639   
1    5      0.20                4001686305409   

                                          STATE  
0  'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'  
1  'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'  
{('4001686153116,4001686340639', "'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'"): [4, 0.2], ('4001686305409', "'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'"): [5, 0.2]}


''

In [44]:
item_list_dict_gsted = item_list_dict
item_list_dict_gsted

{('4001686153116,4001686340639',
  "'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'"): [3.6363636363636362, 0.2],
 ('4001686305409',
  "'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'"): [4.545454545454545, 0.2]}

In [49]:
i = 0
print(classify_promo)
print(promo_id)
if classify_promo.lower() == 'yes':
    for key,value in item_list_dict_gsted.items():
        print(key,value)
        item_code,state = key
        print(item_code,promo_id,clm_start,clm_end,value[0],value[1],state)
        df_each_item = connect_sql(cursor,file_sql = file_sql_summ_state ,item_code = item_code,var_1 = promo_id,var_2 =clm_start,var_3= clm_end,var_4=value[0],var_5 = value[1],var_6 = state)
        if i == 0:
            df_merge = df_each_item
        else :
            df_merge = pd.concat([df_merge, df_each_item], ignore_index=True)
        i+=1
    df_merge['ELI_CLAIM'] = df_merge.RQTY_PROMO * df_merge.SCAN_RATE
    df_merge= df_merge.sort_values(by=['RSKU_ID','RDAY_DT','RSTATE'], ascending=True).reset_index(drop=True)
else:
    for key,value in item_list_dict_gsted.items():
        print(key,value)
        item_code,state = key
        print(item_code,promo_id,clm_start,clm_end,value[0],value[1],state)
        df_each_item = connect_sql(cursor,file_sql = file_sql_summ_state_no_promo ,item_code = item_code,var_1 =clm_start,var_2= clm_end,var_3=value[0],var_4 = value[1],var_5 = state)
        if i == 0:
            df_merge = df_each_item
        else :
            df_merge = pd.concat([df_merge, df_each_item], ignore_index=True)
        i+=1
    df_merge['ELI_CLAIM'] = df_merge.RQTY_PROMO * df_merge.SCAN_RATE
    df_merge= df_merge.sort_values(by=['RSKU_ID','RDAY_DT','RSTATE'], ascending=True).reset_index(drop=True)
df_merge

NO

('4001686153116,4001686340639', "'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'") [3.6363636363636362, 0.2]
4001686153116,4001686340639  2021-08-05 00:00:00 2021-09-01 00:00:00 3.6363636363636362 0.2 'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'
('4001686305409', "'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'") [4.545454545454545, 0.2]
4001686305409  2021-08-05 00:00:00 2021-09-01 00:00:00 4.545454545454545 0.2 'ACT','QLD','SA','NSW','VIC','NT','TAS','WA'


,RDAY_DT,RSKU_ID,RITEM_DESC,RSTATE,RF_NORM_SELL_PRICE_AMT,RPRM_PRICE,RTOTAL_SALES_AMT_MOD,RQTY,RF_ACTUAL_SELL_PRICE_AMT_MOD,RTOTAL_SALES_AMT_NON,RQTY_NON,RF_ACTUAL_SELL_PRICE_AMT_MOD_NON,RTOTAL_SALES_AMT_MOD_PROMO,RQTY_PROMO,RF_ACTUAL_SELL_PRICE_AMT_MOD_1,SCAN_RATE,ELI_CLAIM
0,2021-08-05,4001686153116,HARIBO CONFECTIONERY SOUR S'GHETTI 140G,ACT,4.09,3.64,52.28,23,2.27,0.00,0,0.00,52.28,23,2.27,0.2,4.6
1,2021-08-05,4001686153116,HARIBO CONFECTIONERY SOUR S'GHETTI 140G,NSW,4.09,3.64,575.96,251,2.29,12.27,3,4.09,563.69,248,2.27,0.2,49.6
2,2021-08-05,4001686153116,HARIBO CONFECTIONERY SOUR S'GHETTI 140G,NT,4.09,3.64,59.10,26,2.27,0.00,0,0.00,59.10,26,2.27,0.2,5.2
3,2021-08-05,4001686153116,HARIBO CONFECTIONERY SOUR S'GHETTI 140G,QLD,4.09,3.64,474.59,204,2.33,24.55,6,4.09,450.04,198,2.27,0.2,39.6
4,2021-08-05,4001686153116,HARIBO CONFECTIONERY SOUR S'GHETTI 140G,SA,4.09,3.64,65.92,29,2.27,0.00,0,0.00,65.92,29,2.27,0.2,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2021-09-01,4001686340639,HARIBO SWEET & SOUR BEARS 140G,QLD,4.09,3.64,209.11,92,2.27,0.00,0,0.00,209.11,92,2.27,0.2,18.4
668,2021-09-01,4001686340639,HARIBO SWEET & SOUR BEARS 140G,SA,4.09,3.64,59.09,26,2.27,0.00,0,0.00,59.09,26,2.27,0.2,5.2
669,2021-09-01,4001686340639,HARIBO SWEET & SOUR BEARS 140G,TAS,4.09,3.64,6.82,3,2.27,0.00,0,0.00,6.82,3,2.27,0.2,0.6
670,2021-09-01,4001686340639,HARIBO SWEET & SOUR BEARS 140G,VIC,4.09,3.64,204.10,89,2.29,4.09,1,4.09,200.01,88,2.27,0.2,17.6
